In [8]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from pygrinder import mcar
from pypots.data import load_specific_dataset
from pypots.imputation import SAITS
from pypots.utils.metrics import calc_mae

2024-01-29 17:50:11 [ERROR]: ❌ No module named 'torch_geometric'
Note torch_geometric is missing, please install it with 'pip install torch_geometric torch_scatter torch_sparse' or 'conda install -c pyg pyg pytorch-scatter pytorch-sparse'
2024-01-29 17:50:11 [ERROR]: ❌ name 'MessagePassing' is not defined
Note torch_geometric is missing, please install it with 'pip install torch_geometric torch_scatter torch_sparse' or 'conda install -c pyg pyg pytorch-scatter pytorch-sparse'


In [2]:
import torch

In [3]:
print(torch.__version__)

2.2.0


In [4]:
print(torch.cuda.get_device_properties(0).total_memory)

2089025536


In [9]:

# Data preprocessing. Tedious, but PyPOTS can help.
data = load_specific_dataset('physionet_2012')  # PyPOTS will automatically download and extract it.
X = data['X']
num_samples = len(X['RecordID'].unique())
X = X.drop(['RecordID', 'Time'], axis = 1)
X = StandardScaler().fit_transform(X.to_numpy())
X = X.reshape(num_samples, 48, -1)
X_ori = X  # keep X_ori for validation
X = mcar(X, 0.1)  # randomly hold out 10% observed values as ground truth
dataset = {"X": X}  # X for model input
print(X.shape)  # (11988, 48, 37), 11988 samples, 48 time steps, 37 features

# Model training. This is PyPOTS showtime.
saits = SAITS(n_steps=48, n_features=37, n_layers=2, d_model=256, d_inner=128, n_heads=4, d_k=64, d_v=64, dropout=0.1, epochs=10, device='cuda')
# Here I use the whole dataset as the training set because ground truth is not visible to the model, you can also split it into train/val/test sets
saits.fit(dataset)
imputation = saits.impute(dataset)  # impute the originally-missing values and artificially-missing values
indicating_mask = np.isnan(X) ^ np.isnan(X_ori)  # indicating mask for imputation error calculation
mae = calc_mae(imputation, np.nan_to_num(X_ori), indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)

2024-01-29 17:50:21 [INFO]: Loading the dataset physionet_2012 with TSDB (https://github.com/WenjieDu/Time_Series_Data_Beans)...
2024-01-29 17:50:21 [INFO]: Starting preprocessing physionet_2012...
2024-01-29 17:50:21 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2024-01-29 17:50:21 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2024-01-29 17:50:21 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2024-01-29 17:50:21 [INFO]: Loaded successfully!
2024-01-29 17:50:44 [INFO]: Using the given device: cuda
2024-01-29 17:50:44 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2024-01-29 17:50:44 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 1,378,358


(11988, 48, 37)


2024-01-29 17:51:01 [INFO]: Epoch 001 - training loss: 0.6667
2024-01-29 17:51:16 [INFO]: Epoch 002 - training loss: 0.4856
2024-01-29 17:51:32 [INFO]: Epoch 003 - training loss: 0.4281
2024-01-29 17:51:47 [INFO]: Epoch 004 - training loss: 0.3882
2024-01-29 17:52:03 [INFO]: Epoch 005 - training loss: 0.3702
2024-01-29 17:52:19 [INFO]: Epoch 006 - training loss: 0.3564
2024-01-29 17:52:34 [INFO]: Epoch 007 - training loss: 0.3481
2024-01-29 17:52:50 [INFO]: Epoch 008 - training loss: 0.3388
2024-01-29 17:53:05 [INFO]: Epoch 009 - training loss: 0.3337
2024-01-29 17:53:21 [INFO]: Epoch 010 - training loss: 0.3300
2024-01-29 17:53:21 [INFO]: Finished training.
2024-01-29 17:53:21 [WARNING]: 🚨DeprecationWarning: The method impute is deprecated. Please use `predict` instead.


In [10]:
mae

0.22807315689144705